# Day 9: Non-IID Data Partitioning for FL

**Creating Realistic Heterogeneous Data Distributions**

## Overview
- **Objective**: Simulate realistic non-IID data distributions
- **Challenge**: Real FL systems have heterogeneous data
- **Importance**: Non-IID data significantly impacts FL performance

## What You'll Learn
1. **Types of Non-IID Data**: Label skew, quantity skew, feature skew
2. **Dirichlet Distribution**: Mathematical foundation for partitioning
3. **Visualization**: Understanding heterogeneity visually
4. **Impact on FL**: How non-IID affects model convergence

---

## 1. Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.special import dirichlet

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries imported!")

## 2. Understanding Non-IID Data

**Types of Non-IIDness in FL:**

1. **Label Distribution Skew**: Different class proportions
   - Example: Banks in different regions see different fraud types

2. **Quantity Skew**: Vastly different data amounts
   - Example: Active vs. inactive users

3. **Feature Skew**: Same class, different feature distributions
   - Example: Fraud patterns vary by geography

## 3. Label Skew (Dirichlet Distribution)

In [ ]:
def dirichlet_partition(y, n_clients, alpha, random_state=42):
    """
    Partition data using Dirichlet distribution for label skew.
    
    Args:
        y: Labels array
        n_clients: Number of clients
        alpha: Dirichlet concentration (lower = more skew)
        
    Returns:
        Dictionary mapping client_id to array of indices
    """
    np.random.seed(random_state)
    
    n_classes = len(np.unique(y))
    n_samples = len(y)
    
    # Generate label distribution for each client using Dirichlet
    # p_k ~ Dir(alpha) for each client k
    label_distribution = dirichlet([alpha] * n_classes, size=n_clients)
    
    # Partition indices
    client_indices = {i: [] for i in range(n_clients)}
    
    # For each class, distribute samples according to distribution
    for class_label in range(n_classes):
        # Get indices of samples with this class
        class_indices = np.where(y == class_label)[0]
        np.random.shuffle(class_indices)
        
        # Distribute to clients based on proportions
        proportions = label_distribution[:, class_label]
        
        start_idx = 0
        for client_id in range(n_clients):
            # Number of samples for this client
            n_samples_client = int(len(class_indices) * proportions[client_id])
            
            # Last client gets remaining samples
            if client_id == n_clients - 1:
                n_samples_client = len(class_indices) - start_idx
            
            # Assign samples
            end_idx = min(start_idx + n_samples_client, len(class_indices))
            client_indices[client_id].extend(class_indices[start_idx:end_idx])
            start_idx = end_idx
    
    # Convert to numpy arrays
    for client_id in client_indices:
        client_indices[client_id] = np.array(client_indices[client_id])
    
    return client_indices, label_distribution

# Generate sample data
np.random.seed(42)
n_samples = 10000
n_classes = 5
y = np.random.randint(0, n_classes, n_samples)

print(f"Generated {n_samples} samples with {n_classes} classes")
print(f"Overall class distribution: {np.bincount(y) / n_samples}")

## 4. Compare Different Alpha Values

In [ ]:
n_clients = 10
alphas = [0.1, 0.5, 1.0, 10.0]

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, alpha in enumerate(alphas):
    # Partition
    client_indices, label_dist = dirichlet_partition(y, n_clients, alpha, random_state=42)
    
    # Create matrix: clients x classes
    matrix = np.zeros((n_clients, n_classes))
    for client_id in range(n_clients):
        for class_label in range(n_classes):
            matrix[client_id, class_label] = np.sum(y[client_indices[client_id]] == class_label)
    
    # Normalize rows
    matrix = matrix / (matrix.sum(axis=1, keepdims=True) + 1e-10)
    
    # Plot heatmap
    sns.heatmap(matrix, annot=True, fmt='.2f', cmap='YlOrRd', 
                cbar_kws={'label': 'Proportion'}, ax=axes[idx])
    axes[idx].set_xlabel('Class', fontsize=11)
    axes[idx].set_ylabel('Client', fontsize=11)
    axes[idx].set_title(f'Alpha = {alpha} ({"Extreme" if alpha < 0.5 else "High" if alpha < 2 else "Low"} Skew)', fontsize=12)

plt.tight_layout()
plt.show()

print("\nEffect of Alpha:")
print("  Alpha < 0.5: Extreme skew (each client dominated by 1-2 classes)")
print("  Alpha 0.5-2: High skew (noticeable class imbalance)")
print("  Alpha > 5: Low skew (approaches IID)")

## 5. Quantity Skew (Power Law Distribution)

In [ ]:
def quantity_skew_partition(y, n_clients, exponent, random_state=42):
    """
    Partition data with quantity skew following power law.
    
    Args:
        y: Labels array
        n_clients: Number of clients
        exponent: Power law exponent (lower = more skew)
        
    Returns:
        Dictionary mapping client_id to array of indices
    """
    np.random.seed(random_state)
    
    n_samples = len(y)
    
    # Generate client sizes following Pareto distribution
    # P(x) ~ x^(-exponent)
    client_ranks = np.arange(1, n_clients + 1)
    sizes_raw = client_ranks ** (-exponent)
    sizes = (sizes_raw / sizes_raw.sum() * n_samples).astype(int)
    
    # Adjust to sum to n_samples
    sizes[-1] += n_samples - sizes.sum()
    
    # Shuffle data and assign
    indices = np.random.permutation(n_samples)
    client_indices = {}
    start_idx = 0
    
    for client_id in range(n_clients):
        end_idx = start_idx + sizes[client_id]
        client_indices[client_id] = indices[start_idx:end_idx]
        start_idx = end_idx
    
    return client_indices, sizes

# Compare different exponents
exponents = [1.2, 2.0, 3.0]

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, exponent in enumerate(exponents):
    client_indices, sizes = quantity_skew_partition(y, n_clients, exponent, random_state=42)
    
    # Plot
    axes[idx].bar(range(n_clients), sizes, color='steelblue', alpha=0.7)
    axes[idx].set_xlabel('Client', fontsize=11)
    axes[idx].set_ylabel('Sample Count', fontsize=11)
    axes[idx].set_title(f'Exponent = {exponent} ({"Extreme" if exponent < 1.5 else "High" if exponent < 2.5 else "Moderate"} Skew)', fontsize=12)
    axes[idx].grid(True, alpha=0.3, axis='y')
    
    # Add statistics
    top_10_pct = int(0.1 * n_clients)
    top_10_share = sizes[:top_10_pct].sum() / sizes.sum() * 100
    axes[idx].text(0.5, 0.95, f'Top 10% have {top_10_share:.1f}% of data',
                 transform=axes[idx].transAxes, ha='center', va='top',
                 bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

print("\nEffect of Exponent:")
print("  Exponent 1.1-1.3: Extreme skew (top 10% have 70%+ of data)")
print("  Exponent 1.5-2.0: High skew (top 20% have 50%+ of data)")
print("  Exponent 2.5+: Moderate skew (more uniform distribution)")

## 6. Feature Skew (Clustering-Based)

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans

# Generate data with clusters
X_clusters, y_clusters = make_blobs(
    n_samples=5000,
    n_features=2,
    centers=5,
    cluster_std=2.0,
    random_state=42
)

# Add labels (simplified: 2 classes based on position)
y_binary = (X_clusters[:, 0] + X_clusters[:, 1] > 0).astype(int)

# Cluster-based partition
n_clusters = 10
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(X_clusters)

# Assign clusters to clients
client_indices = {i: np.where(cluster_labels == i)[0] for i in range(n_clusters)}

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Data clusters
scatter = axes[0].scatter(X_clusters[:, 0], X_clusters[:, 1], 
                          c=cluster_labels, cmap='tab10', alpha=0.6)
axes[0].set_xlabel('Feature 1', fontsize=12)
axes[0].set_ylabel('Feature 2', fontsize=12)
axes[0].set_title('Feature Clusters (Geographic Segmentation)', fontsize=14)
plt.colorbar(scatter, ax=axes[0], label='Cluster')

# Plot 2: Class distribution per cluster
matrix = np.zeros((n_clusters, 2))
for client_id in range(n_clusters):
    indices = client_indices[client_id]
    matrix[client_id, 0] = np.sum(y_binary[indices] == 0)
    matrix[client_id, 1] = np.sum(y_binary[indices] == 1)

matrix_norm = matrix / (matrix.sum(axis=1, keepdims=True) + 1e-10)

sns.heatmap(matrix_norm, annot=True, fmt='.2f', cmap='RdBu_r',
            xticklabels=['Class 0', 'Class 1'],
            yticklabels=[f'Client {i}' for i in range(n_clusters)],
            cbar_kws={'label': 'Proportion'}, ax=axes[1])
axes[1].set_title('Class Distribution per Client', fontsize=14)
axes[1].set_xlabel('Class', fontsize=12)
axes[1].set_ylabel('Client', fontsize=12)

plt.tight_layout()
plt.show()

print("\nFeature Skew:")
print("  Clients in different regions have different class distributions")
print("  Even with balanced labels, feature space causes heterogeneity")

## 7. Heterogeneity Metrics

In [ ]:
def compute_heterogeneity_metrics(client_indices, y, n_classes):
    """
    Compute metrics to quantify data heterogeneity.
    """
    n_clients = len(client_indices)
    
    # 1. Label entropy per client
    entropies = []
    for client_id in range(n_clients):
        indices = client_indices[client_id]
        client_labels = y[indices]
        
        # Compute class distribution
        class_counts = np.bincount(client_labels, minlength=n_classes)
        probs = class_counts / class_counts.sum()
        
        # Entropy (only non-zero probabilities)
        probs = probs[probs > 0]
        entropy = -np.sum(probs * np.log(probs))
        entropies.append(entropy)
    
    # 2. Sample size statistics
    sizes = np.array([len(client_indices[i]) for i in range(n_clients)])
    
    # 3. Gini coefficient (inequality in sizes)
    sorted_sizes = np.sort(sizes)
    n = len(sorted_sizes)
    gini = (2 * np.sum(np.arange(1, n + 1) * sorted_sizes) - (n + 1) * sorted_sizes.sum()) / (n * sorted_sizes.sum())
    
    return {
        'mean_entropy': np.mean(entropies),
        'std_entropy': np.std(entropies),
        'mean_size': sizes.mean(),
        'std_size': sizes.std(),
        'min_size': sizes.min(),
        'max_size': sizes.max(),
        'max_min_ratio': sizes.max() / (sizes.min() + 1e-10),
        'gini_coefficient': gini
    }

# Compare partitions
comparisons = [
    ('IID', dirichlet_partition(y, n_clients, alpha=100, random_state=42)[0]),
    ('Label Skew (α=0.5)', dirichlet_partition(y, n_clients, alpha=0.5, random_state=42)[0]),
    ('Quantity Skew (exp=1.5)', quantity_skew_partition(y, n_clients, exponent=1.5, random_state=42)[0]),
]

print("\n" + "="*70)
print("HETEROGENEITY METRICS COMPARISON")
print("="*70)

for name, indices in comparisons:
    metrics = compute_heterogeneity_metrics(indices, y, n_classes)
    print(f"\n{name}:")
    print(f"  Mean label entropy:  {metrics['mean_entropy']:.3f} ± {metrics['std_entropy']:.3f}")
    print(f"  Mean client size:    {metrics['mean_size']:.0f} ± {metrics['std_size']:.0f}")
    print(f"  Max/min ratio:      {metrics['max_min_ratio']:.2f}")
    print(f"  Gini coefficient:   {metrics['gini_coefficient']:.3f}")

## 8. Summary

### Non-IID Data Types:

1. **Label Skew** (Dirichlet distribution):
   - Different class proportions per client
   - Controlled by alpha parameter
   - α < 0.5: Extreme skew

2. **Quantity Skew** (Power law):
   - Different data amounts per client
   - Follows real-world distributions
   - Exponent 1.5: Top 20% have 50%+ of data

3. **Feature Skew** (Clustering):
   - Same class, different feature distributions
   - Geographic or demographic segmentation

### Key Takeaways:

- ✅ **Non-IID data is realistic**: Real FL systems face heterogeneity
- ✅ **Impacts performance**: Non-IID slows convergence, reduces accuracy
- ✅ **Quantifiable**: Entropy, Gini coefficient, max/min ratio
- ✅ **Alpha is key**: Dirichlet α controls skew level

### Why Non-IID Matters:

- **Convergence**: Non-IID data slows FL convergence
- **Accuracy**: Final model accuracy can be significantly lower
- **Fairness**: Some clients may be poorly represented
- **Personalization**: Opportunity for client-specific models

### Next Steps:
→ **Day 10**: Flower FL Framework (production FL)
→ **Day 20**: Federated Personalization (handle non-IID)

---

**📁 Project Location**: `02_federated_learning_foundations/non_iid_partitioner/`